## ☕ Coffee Sales Dataset Schema

This table describes the columns (features) found in the `Coffe_sales.csv` file, their data types, and their purpose.

| Feature Name | Description | Data Type / Format |
| :--- | :--- | :--- |
| **`hour_of_day`** | Hour of purchase | Integer (0–23) |
| **`cash_type`** | Mode of payment | Categorical (cash / card) |
| **`money`** | Transaction amount | Numeric (local currency) |
| **`coffee_name`** | Type of coffee purchased | String (e.g., Latte, Americano) |
| **`Time_of_Day`** | Categorized time of purchase | Categorical (Morning, Afternoon, Night) |
| **`Weekday`** | Day of the week | String (e.g., Mon, Tue, ...) |
| **`Month_name`** | Month of purchase | String (e.g., Jan, Feb, Mar) |
| **`Weekdaysort`** | Numeric representation for weekday ordering | Integer (1 = Mon, 7 = Sun) |
| **`Monthsort`** | Numeric representation for month ordering | Integer (1 = Jan, 12 = Dec) |
| **`Date`** | Date of transaction | Date (YYYY-MM-DD) |
| **`Time`** | Exact time of transaction | Time (HH:MM:SS) |

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine, inspect
from injest_db import engine

In [2]:
inspector = inspect(engine)

table_names = inspector.get_table_names()

print(f"Tables found in the database: {table_names}")

Tables found in the database: ['sales']


# BUSINESS OVERVIEW & KEY METRICS

In [ ]:
# Executive Summary Dashboard

pd.read_sql_query('''SELECT 
    COUNT(*) AS total_transactions,
    ROUND(SUM(money), 2) AS total_revenue,
    ROUND(AVG(money), 2) AS avg_transaction_value,
    COUNT(DISTINCT Date) AS operating_days,
    COUNT(DISTINCT coffee_name) AS unique_products,
    MIN(Date) AS first_sale,
    MAX(Date) AS last_sale,
    ROUND(SUM(money) / COUNT(DISTINCT Date), 2) AS avg_daily_revenue,
    ROUND(COUNT(*) / COUNT(DISTINCT Date), 2) AS avg_daily_transactions
FROM sales;''', engine)

,total_transactions,total_revenue,avg_transaction_value,operating_days,unique_products,first_sale,last_sale,avg_daily_revenue,avg_daily_transactions
0,3547,112245.58,31.65,381,8,2024-03-01,2025-03-23,294.61,9.31


In [14]:
# Monthly Revenue Trend

pd.read_sql_query('''SELECT
    DATE_FORMAT(Date, '%Y-%m') AS month_year,
    month_name, -- Enclose in backticks
    COUNT(*) AS transactions,
    ROUND(SUM(money), 2) AS revenue,
    ROUND(AVG(money), 2) AS avg_ticket,
    COUNT(DISTINCT Date) AS days_open,
    ROUND(SUM(money) / COUNT(DISTINCT Date), 2) AS avg_daily_revenue
FROM sales
GROUP BY DATE_FORMAT(Date, '%Y-%m'), month_name, monthsort -- Also in GROUP BY
ORDER BY DATE_FORMAT(Date, '%Y-%m');''', engine)

,month_year,month_name,transactions,revenue,avg_ticket,days_open,avg_daily_revenue
0,2024-03,Mar,175,5905.20,33.74,31,190.49
1,2024-04,Apr,168,5719.56,34.05,30,190.65
2,2024-05,May,241,8164.42,33.88,28,291.59
3,2024-06,Jun,223,7617.76,34.16,30,253.93
4,2024-07,Jul,237,6915.94,29.18,31,223.09
5,2024-08,Aug,272,7613.84,27.99,31,245.61
6,2024-09,Sep,344,9988.64,29.04,30,332.95
7,2024-10,Oct,426,13891.16,32.61,31,448.10
8,2024-11,Nov,259,8590.54,33.17,29,296.23
9,2024-12,Dec,259,8237.74,31.81,31,265.73


In [18]:
# Month-over-Month Growth Analysis

pd.read_sql_query('''WITH monthly_stats AS (
    SELECT 
        DATE_FORMAT(Date, '%Y-%m') AS month_year,
        SUM(money) AS revenue,
        COUNT(*) AS transactions
    FROM sales
    GROUP BY month_year
)
SELECT 
    month_year,
    revenue,
    transactions,
    LAG(revenue) OVER (ORDER BY month_year) AS prev_month_revenue,
    ROUND(((revenue - LAG(revenue) OVER (ORDER BY month_year)) / 
           LAG(revenue) OVER (ORDER BY month_year) * 100), 2) AS revenue_growth_pct,
    transactions - LAG(transactions) OVER (ORDER BY month_year) AS transaction_change
FROM monthly_stats
ORDER BY month_year;''', engine)

,month_year,revenue,transactions,prev_month_revenue,revenue_growth_pct,transaction_change
0,2024-03,5905.20,175,NaN,NaN,NaN
1,2024-04,5719.56,168,5905.20,-3.14,-7.0
2,2024-05,8164.42,241,5719.56,42.75,73.0
3,2024-06,7617.76,223,8164.42,-6.70,-18.0
4,2024-07,6915.94,237,7617.76,-9.21,14.0
5,2024-08,7613.84,272,6915.94,10.09,35.0
6,2024-09,9988.64,344,7613.84,31.19,72.0
7,2024-10,13891.16,426,9988.64,39.07,82.0
8,2024-11,8590.54,259,13891.16,-38.16,-167.0
9,2024-12,8237.74,259,8590.54,-4.11,0.0


# PRODUCT PERFORMANCE ANALYSIS

In [19]:
# Best Selling Products (By Quantity & Revenue)

pd.read_sql_query('''SELECT 
    coffee_name,
    COUNT(*) AS units_sold,
    ROUND(SUM(money), 2) AS total_revenue,
    ROUND(AVG(money), 2) AS avg_price,
    ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM sales), 2) AS pct_of_sales,
    ROUND(SUM(money) * 100.0 / (SELECT SUM(money) FROM sales), 2) AS pct_of_revenue
FROM sales
GROUP BY coffee_name
ORDER BY total_revenue DESC;''', engine)

,coffee_name,units_sold,total_revenue,avg_price,pct_of_sales,pct_of_revenue
0,Latte,757,26875.30,35.50,21.34,23.94
1,Americano with Milk,809,24751.12,30.59,22.81,22.05
2,Cappuccino,486,17439.14,35.88,13.70,15.54
3,Americano,564,14650.26,25.98,15.90,13.05
4,Hot Chocolate,276,9933.46,35.99,7.78,8.85
5,Cocoa,239,8521.16,35.65,6.74,7.59
6,Cortado,287,7384.86,25.73,8.09,6.58
7,Espresso,129,2690.28,20.85,3.64,2.40


In [20]:
# Product Performance by Time of Day

pd.read_sql_query('''SELECT 
    time_of_day,
    coffee_name,
    COUNT(*) AS quantity_sold,
    ROUND(SUM(money), 2) AS revenue,
    ROUND(AVG(money), 2) AS avg_price
FROM sales
GROUP BY time_of_day, coffee_name
ORDER BY time_of_day, quantity_sold DESC;''', engine)

,time_of_day,coffee_name,quantity_sold,revenue,avg_price
0,Afternoon,Latte,270,9611.10,35.60
1,Afternoon,Americano with Milk,239,7384.36,30.90
2,Afternoon,Americano,233,6133.94,26.33
3,Afternoon,Cappuccino,164,5910.70,36.04
4,Afternoon,Cortado,88,2315.84,26.32
5,Afternoon,Hot Chocolate,80,2899.02,36.24
6,Afternoon,Cocoa,75,2685.92,35.81
7,Afternoon,Espresso,56,1189.16,21.23
8,Morning,Americano with Milk,331,10025.52,30.29
9,Morning,Americano,219,5643.10,25.77


In [21]:
# Top 3 Products per Time Period
pd.read_sql_query('''WITH ranked_products AS (
    SELECT 
        time_of_day,
        coffee_name,
        COUNT(*) AS quantity,
        ROUND(SUM(money), 2) AS revenue,
        ROW_NUMBER() OVER (PARTITION BY time_of_day ORDER BY COUNT(*) DESC) AS rank_pos
    FROM sales
    GROUP BY time_of_day, coffee_name
)
SELECT 
    time_of_day,
    coffee_name,
    quantity,
    revenue
FROM ranked_products
WHERE rank_pos <= 3
ORDER BY time_of_day, rank_pos;''', engine)

,time_of_day,coffee_name,quantity,revenue
0,Afternoon,Latte,270,9611.10
1,Afternoon,Americano with Milk,239,7384.36
2,Afternoon,Americano,233,6133.94
3,Morning,Americano with Milk,331,10025.52
4,Morning,Americano,219,5643.10
5,Morning,Latte,215,7650.18
6,Night,Latte,272,9614.02
7,Night,Americano with Milk,239,7341.24
8,Night,Cappuccino,200,7201.00


# TIME-BASED ANALYSIS

In [23]:
# Hourly Sales Pattern (Peak Hours Analysis)

pd.read_sql_query('''SELECT 
    hour_of_day,
    COUNT(*) AS transactions,
    ROUND(SUM(money), 2) AS revenue,
    ROUND(AVG(money), 2) AS avg_ticket,
    ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM sales), 2) AS pct_of_total
FROM sales
GROUP BY hour_of_day
ORDER BY hour_of_day;''',engine)

,hour_of_day,transactions,revenue,avg_ticket,pct_of_total
0,6,5,149.40,29.88,0.14
1,7,88,2846.02,32.34,2.48
2,8,235,7017.88,29.86,6.63
3,9,242,7264.28,30.02,6.82
4,10,328,10198.52,31.09,9.25
5,11,283,8453.10,29.87,7.98
6,12,241,7419.62,30.79,6.79
7,13,225,7028.76,31.24,6.34
8,14,225,7173.80,31.88,6.34
9,15,236,7476.02,31.68,6.65


In [25]:
# Peak Hours Identification
# dynamic classification metrics based on the average hourly transaction volume.

pd.read_sql_query('''SELECT
    hour_of_day,
    COUNT(*) AS transactions,
    -- Calculates the total revenue for the hour, rounded to two decimal places
    ROUND(SUM(money), 2) AS revenue,
    -- Classifies the hour based on how far its transaction count is from the overall average
    CASE
        -- Peak Hour: Transaction count is 50% or more above the average
        WHEN COUNT(*) >= (SELECT AVG(cnt) * 1.5 FROM (SELECT COUNT(*) AS cnt FROM sales GROUP BY hour_of_day) AS t)
        THEN 'Peak Hour'
        -- Normal Hour: Transaction count is at or above the average
        WHEN COUNT(*) >= (SELECT AVG(cnt) FROM (SELECT COUNT(*) AS cnt FROM sales GROUP BY hour_of_day) AS t)
        THEN 'Normal Hour'
        -- Slow Hour: Transaction count is below the average
        ELSE 'Slow Hour'
    END AS hour_classification
FROM
    sales
GROUP BY
    hour_of_day
ORDER BY
    transactions DESC;''', engine)

,hour_of_day,transactions,revenue,hour_classification
0,10,328,10198.52,Peak Hour
1,11,283,8453.10,Normal Hour
2,16,278,9031.84,Normal Hour
3,9,242,7264.28,Normal Hour
4,12,241,7419.62,Normal Hour
5,17,237,7659.76,Normal Hour
6,15,236,7476.02,Normal Hour
7,8,235,7017.88,Normal Hour
8,19,229,7751.96,Normal Hour
9,13,225,7028.76,Normal Hour


In [30]:
# Day of Week Performance
pd.read_sql_query('''SELECT 
    weekday,
    weekdaysort,
    COUNT(*) AS transactions,
    ROUND(SUM(money), 2) AS revenue,
    ROUND(AVG(money), 2) AS avg_ticket,
    ROUND(COUNT(*) / COUNT(DISTINCT Date), 2) AS avg_transactions_per_day
FROM sales
GROUP BY weekday, weekdaysort
ORDER BY weekdaysort;''', engine)

,weekday,weekdaysort,transactions,revenue,avg_ticket,avg_transactions_per_day
0,Mon,1,544,17363.10,31.92,9.89
1,Tue,2,572,18168.38,31.76,10.40
2,Wed,3,500,15750.46,31.50,9.62
3,Thu,4,510,16091.40,31.55,9.44
4,Fri,5,532,16802.66,31.58,9.50
5,Sat,6,470,14733.52,31.35,8.55
6,Sun,7,419,13336.06,31.83,7.76


In [32]:
# Weekend vs Weekday Comparison

pd.read_sql_query('''SELECT 
    CASE 
        WHEN weekday IN ('Sat', 'Sun') THEN 'Weekend'
        ELSE 'Weekday'
    END AS day_type,
    COUNT(*) AS transactions,
    ROUND(SUM(money), 2) AS revenue,
    ROUND(AVG(money), 2) AS avg_ticket,
    COUNT(DISTINCT Date) AS days,
    ROUND(SUM(money) / COUNT(DISTINCT Date), 2) AS avg_daily_revenue
FROM sales
GROUP BY CASE WHEN weekday IN ('Sat', 'Sun') THEN 'Weekend' ELSE 'Weekday' END;''', engine)

,day_type,transactions,revenue,avg_ticket,days,avg_daily_revenue
0,Weekday,2658,84176.00,31.67,272,309.47
1,Weekend,889,28069.58,31.57,109,257.52


# DAILY PERFORMANCE ANALYSIS

In [34]:
# Best Performing Days

pd.read_sql_query('''SELECT 
    Date,
    weekday,
    COUNT(*) AS transactions,
    ROUND(SUM(money), 2) AS revenue,
    ROUND(AVG(money), 2) AS avg_ticket
FROM sales
GROUP BY Date, weekday
ORDER BY revenue DESC
LIMIT 20;''', engine)

,Date,weekday,transactions,revenue,avg_ticket
0,2024-10-11,Fri,26,836.66,32.18
1,2025-02-03,Mon,25,796.00,31.84
2,2025-02-05,Wed,24,760.24,31.68
3,2025-03-03,Mon,22,737.72,33.53
4,2024-10-17,Thu,21,726.46,34.59
5,2025-02-24,Mon,22,723.02,32.86
6,2025-03-14,Fri,23,704.88,30.65
7,2025-02-21,Fri,23,695.08,30.22
8,2024-09-22,Sun,24,679.88,28.33
9,2025-02-10,Mon,21,662.76,31.56


In [37]:
# Worst Performing Days

pd.read_sql_query('''SELECT 
    Date,
    weekday,
    COUNT(*) AS transactions,
    ROUND(SUM(money), 2) AS revenue,
    ROUND(AVG(money), 2) AS avg_ticket
FROM sales
GROUP BY Date, weekday
ORDER BY revenue ASC
LIMIT 20;''', engine)

,Date,weekday,transactions,revenue,avg_ticket
0,2024-08-29,Thu,1,23.02,23.02
1,2025-01-06,Mon,1,25.96,25.96
2,2024-07-17,Wed,1,27.92,27.92
3,2024-03-17,Sun,1,28.90,28.90
4,2024-08-27,Tue,1,32.82,32.82
5,2024-05-13,Mon,1,32.82,32.82
6,2024-07-15,Mon,1,32.82,32.82
7,2024-11-24,Sun,1,35.76,35.76
8,2024-04-27,Sat,1,37.72,37.72
9,2024-05-08,Wed,1,37.72,37.72


# ADVANCED BUSINESS INSIGHTS

In [40]:
# Sales Velocity (Transactions per Hour by Day)

pd.read_sql_query('''SELECT 
    Date,
    weekday,
    COUNT(*) AS total_transactions,
    COUNT(DISTINCT hour_of_day) AS hours_operated,
    ROUND(COUNT(*) / COUNT(DISTINCT hour_of_day), 2) AS transactions_per_hour
FROM sales
GROUP BY Date, weekday
ORDER BY transactions_per_hour DESC
LIMIT 20;''', engine)

,Date,weekday,total_transactions,hours_operated,transactions_per_hour
0,2025-03-12,Wed,17,5,3.40
1,2024-08-20,Tue,13,4,3.25
2,2024-03-09,Sat,13,4,3.25
3,2024-08-24,Sat,6,2,3.00
4,2024-05-06,Mon,9,3,3.00
5,2024-10-04,Fri,9,3,3.00
6,2024-09-21,Sat,15,5,3.00
7,2024-05-25,Sat,6,2,3.00
8,2024-07-01,Mon,3,1,3.00
9,2025-02-14,Fri,15,5,3.00


In [41]:
# Revenue Concentration Analysis (Pareto 80/20)

pd.read_sql_query('''WITH product_revenue AS (
    SELECT 
        coffee_name,
        SUM(money) AS revenue,
        ROUND(SUM(money) * 100.0 / (SELECT SUM(money) FROM sales), 2) AS revenue_pct
    FROM sales
    GROUP BY coffee_name
    ORDER BY revenue DESC
),
cumulative AS (
    SELECT 
        coffee_name,
        revenue,
        revenue_pct,
        SUM(revenue_pct) OVER (ORDER BY revenue DESC) AS cumulative_pct
    FROM product_revenue
)
SELECT 
    coffee_name,
    revenue,
    revenue_pct,
    cumulative_pct,
    CASE 
        WHEN cumulative_pct <= 80 THEN 'Top 80% Revenue'
        ELSE 'Bottom 20% Revenue'
    END AS pareto_class
FROM cumulative
ORDER BY revenue DESC;''', engine)

,coffee_name,revenue,revenue_pct,cumulative_pct,pareto_class
0,Latte,26875.30,23.94,23.94,Top 80% Revenue
1,Americano with Milk,24751.12,22.05,45.99,Top 80% Revenue
2,Cappuccino,17439.14,15.54,61.53,Top 80% Revenue
3,Americano,14650.26,13.05,74.58,Top 80% Revenue
4,Hot Chocolate,9933.46,8.85,83.43,Bottom 20% Revenue
5,Cocoa,8521.16,7.59,91.02,Bottom 20% Revenue
6,Cortado,7384.86,6.58,97.60,Bottom 20% Revenue
7,Espresso,2690.28,2.40,100.00,Bottom 20% Revenue


In [42]:
# Seasonal Patterns

pd.read_sql_query('''SELECT 
    CASE 
        WHEN monthsort IN (12, 1, 2) THEN 'Winter'
        WHEN monthsort IN (3, 4, 5) THEN 'Spring'
        WHEN monthsort IN (6, 7, 8) THEN 'Summer'
        ELSE 'Fall'
    END AS season,
    COUNT(*) AS transactions,
    ROUND(SUM(money), 2) AS revenue,
    ROUND(AVG(money), 2) AS avg_ticket
FROM sales
GROUP BY season
ORDER BY FIELD(season, 'Spring', 'Summer', 'Fall', 'Winter');''', engine)

,season,transactions,revenue,avg_ticket
0,Spring,903,29775.62,32.97
1,Summer,732,22147.54,30.26
2,Fall,1029,32470.34,31.56
3,Winter,883,27852.08,31.54


In [45]:
# 7-Day Moving Average (Recent Trend)

pd.read_sql_query('''WITH daily_sales AS (
    SELECT 
        Date,
        SUM(money) AS daily_revenue,
        COUNT(*) AS daily_transactions
    FROM sales
    GROUP BY Date
)
SELECT 
    Date,
    daily_revenue,
    daily_transactions,
    ROUND(AVG(daily_revenue) OVER (
        ORDER BY Date 
        ROWS BETWEEN 6 PRECEDING AND CURRENT ROW
    ), 2) AS ma_7day_revenue,
    ROUND(AVG(daily_transactions) OVER (
        ORDER BY Date 
        ROWS BETWEEN 6 PRECEDING AND CURRENT ROW
    ), 2) AS ma_7day_transactions
FROM daily_sales
ORDER BY Date DESC
LIMIT 30;''', engine)

,Date,daily_revenue,daily_transactions,ma_7day_revenue,ma_7day_transactions
0,2025-03-23,204.76,6,482.63,15.57
1,2025-03-22,365.42,12,476.83,15.43
2,2025-03-21,636.80,20,456.88,14.71
3,2025-03-20,597.60,20,466.61,15.14
4,2025-03-19,623.56,21,433.44,14.00
5,2025-03-18,549.12,17,422.11,13.43
6,2025-03-17,401.18,13,403.77,12.86
7,2025-03-16,164.10,5,425.61,13.43
8,2025-03-15,225.82,7,436.73,13.86
9,2025-03-14,704.88,23,437.64,14.00
